In [ ]:
#Welcome to Ali's Notebook

In [1]:
#Imports
import pandas as pd
import numpy as np
from google.oauth2 import service_account
import pandas_gbq

In [2]:
credentials = service_account.Credentials.from_service_account_file(
    'service_key.json',
)

df = pandas_gbq.read_gbq("select * from cfpb_complaints.complaint_database",dialect='standard',project_id="my-ds-projects",use_bqstorage_api=True,credentials=credentials)
df

Downloading: 100%|█████████████████████████████████████████████████████████████|


,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2019-11-09,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,None,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,33169.0,None,Consent not provided,Web,2019-11-09,Closed with non-monetary relief,True,<NA>,3434182
1,2018-02-13,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,None,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,TN,38053.0,None,Consent not provided,Web,2018-02-13,Closed with non-monetary relief,True,<NA>,2813841
2,2020-10-06,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,I have disputed the inaccurate reportings with...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,GA,30058.0,None,Consent provided,Web,2020-10-06,Closed with explanation,True,<NA>,3884040
3,2021-04-02,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,I went to get financing for a car loan. I was ...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,GA,30312.0,None,Consent provided,Web,2021-04-02,Closed with non-monetary relief,True,<NA>,4267194
4,2020-07-21,Mortgage,Conventional home mortgage,Trouble during payment process,None,Refinancing loan signed and closed on XX/XX/XX...,Company believes it acted appropriately as aut...,CrossCountry Mortgage LLC,FL,33713.0,None,Consent provided,Web,2020-07-21,Closed with explanation,True,<NA>,3755712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458901,2012-08-01,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",None,None,None,WELLS FARGO & COMPANY,UT,84401,None,None,Web,2012-08-08,Closed with monetary relief,True,True,128308
3458902,2022-12-09,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Fraud or scam,None,None,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,OR,97117.0,"Older American, Servicemember",Consent not provided,Web,2022-12-09,Closed with explanation,True,<NA>,6301991
3458903,2014-04-29,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",None,None,None,WELLS FARGO & COMPANY,NY,11530,None,None,Web,2014-05-02,Closed with explanation,True,False,828975
3458904,2022-04-06,Mortgage,Conventional home mortgage,Trouble during payment process,None,None,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,MI,48070.0,None,Consent not provided,Web,2022-04-06,Closed with non-monetary relief,True,<NA>,5414488


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=14312678-500f-4667-acd9-8d9c457ddf09' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>